In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.stats import scoreatpercentile
import pandas as pd
%matplotlib inline


Statistical difference in conversion

In [2]:
def simulate_test(success, trials, N=1):
    return np.random.beta(success + 1, trials - success + 1, size=N)

In [3]:
def run_split_test(Ax, An, Bx, Bn, N=10000):
    As = simulate_test(Ax, An, N)
    Bs = simulate_test(Bx, Bn, N)
    D = Bs - As
    pBeta = (D > 0).mean()
    d1 = (scoreatpercentile(D, 2.5))
    d2 = (scoreatpercentile(D, 50))
    d3 = (scoreatpercentile(D, 97.5))
    return pd.DataFrame({'2.5':[d1],'50':[d2], '97.5' : [d3]})

In [4]:
run_split_test(111, 1198, 123, 1198)

,2.5,50,97.5
0,-0.013241,0.009894,0.03463


Statistical difference in margin

In [5]:
#replace lgw_values.csv with your csv file name in function (get_avg_margin) to import margin per booking 
#this should be a list of margin per bookings by ref over a reasonable period depending on the amount of data available
def get_avg_margin(filename):
    dfA = pd.read_csv(filename, sep=',', encoding='latin1')
    avg_margin = dfA.ADDON_MARGIN.values
    avg_margin = avg_margin[avg_margin > 0]
    return avg_margin

avg_margin = get_avg_margin('lgw_value.csv')

In [6]:
#WIP - haven't got this to work quite right yet

def get_margin_range(totA, totB, conversionsA, conversionsB, sessionsA, sessionsB):
    conversionA_beta = np.random.beta(conversionsA, sessionsA)
    bookingsA = np.random.binomial(1, conversionA_beta, totA)
    revenueA = np.random.choice(avg_margin, bookingsA.sum(), replace=True)
    revenueA_sum = revenueA.sum()
    conversionB_beta = np.random.beta(conversionsB, sessionsB)
    bookingsB = np.random.binomial(1, conversionB_beta, totB)
    revenueB = np.random.choice(avg_margin, bookingsB.sum(), replace=True)
    revenueB_sum = revenueB.sum()
    test_ests = np.array([revenueB_sum for _ in range(10000)])
    control_ests = np.array([revenueA_sum for _ in range(10000)])
    delta = test_ests - control_ests
    print (delta > 0).mean(), control_ests.mean(), scoreatpercentile(delta, 2.5), scoreatpercentile(delta, 50), scoreatpercentile(delta, 97.5)

    

In [7]:
get_margin_range(54659, 54659, 121, 136, 1459, 1459)

1.0 48168.17 5931.74 5931.74 5931.74


In [13]:
def get_test_estimate(tot, conversions, sessions):
    parking_conversion_beta = np.random.beta(conversions, sessions)
    parking_bookings = np.random.binomial(1, parking_conversion_beta, tot)
    parking_revenue = np.random.choice(avg_margin, parking_bookings.sum(), replace=True)
    return parking_revenue.sum()


def get_control_estimate(tot, conversions, sessions):
    conversion_beta = np.random.beta(conversions, sessions)
    bookings = np.random.binomial(1, conversion_beta, tot)
    revenue = np.random.choice(avg_margin, bookings.sum(), replace=True)
    return revenue.sum()

test_ests = np.array([get_test_estimate(54659, 121, 1459) for _ in range(10000)])
control_ests = np.array([get_control_estimate(54659, 136, 1459) for _ in range(10000)])

In [12]:
delta = test_ests - control_ests
print((delta > 0).mean())
print(control_ests.mean(), 
      #range of the deltas difference between test and control value
      scoreatpercentile(delta, 2.5), 
      scoreatpercentile(delta, 50), 
      scoreatpercentile(delta, 97.5))

0.1851
(47483.464869000003, -15511.693749999997, -4846.9500000000007, 6166.6280000000006)


In [ ]:
delta = test_ests - control_ests
print((delta > 0).mean())
print(control_ests.mean(), 
      #range of the deltas difference between test and control value
      scoreatpercentile(delta, 2.5), 
      scoreatpercentile(delta, 50), 
      scoreatpercentile(delta, 97.5))